In [658]:
import pandas as pd
import numpy as np

df = pd.read_csv('pre_processed.csv')


In [659]:
def fill_deck_cabin_side(row):
    # Check if Number is NA and Cabin is not NA
    if pd.isna(row['Number']) and not pd.isna(row['Cabin']):
        # Split the Cabin column
        split = row['Cabin'].split('/')
        # Return a Series containing the split values correctly cast
        return pd.Series([split[0], int(split[1]), split[2]])
    else:
        # Return the current values, this might need adjustment based on your needs
        return pd.Series([row.Deck, row.Number, row.Side])

In [660]:
decks_by_planet = {
    'Earth':['E','F','G'],
    'Europa': ['A','B','C','D','E','T'],
    'Mars': ['D','E','F']
}

decks_by_planet_no_bills = {
    'Earth':['G'],
    'Europa':['B'],
    'Mars': ['E','F']
}

planet_by_deck = {
    'A':['Europa'],'B':['Europa'],'C':['Europa'],'D':['Europa','Mars'],'E':['Europa','Mars','Earth'],
    'F':['Earth','Mars'],'G':['Earth'],'T':['Europa']
}

homeplanets = ['Earth', 'Europa', 'Mars']

cabin_sides = ['P','S']

cabin_decks = list(df.dropna(subset = ['Deck']).Deck.unique())


In [661]:
def multiple_decks_in_group(df,row):
    return len(df[df.Group == row.Group].dropna(subset ='Deck').Deck.unique()) > 1
        

In [662]:
def fill_potential_decks(row):
    if pd.isna(row.Cabin):
        if row.Bills == 0 and not pd.isna(row.HomePlanet):
            if len(df[df.Group == row.Group].dropna(subset = 'Deck').Deck.unique()) > 1:
                return decks_by_planet_no_bills[row.HomePlanet]
        if not pd.isna(row.HomePlanet):
            return decks_by_planet[row.HomePlanet]
        else:
            return ['A','B','C','D','E','F','G','T']
df['potential_decks'] = df.apply(fill_potential_decks, axis = 1)

In [663]:
type(df.iloc[15].potential_decks)

list

# Filling missed Cabins


In [664]:
def utilised_cabins(df):
    used_cabins = {}
    for deck in list(df.dropna().Deck.unique()):
        cab = {}
        for side in list(df.dropna().Side.unique()):
            rooms = list(df[(df.Deck == deck) & (df.Side == side)].dropna().Number.unique())
            rooms.sort()
            cab[side] = rooms
        used_cabins[deck] = cab
    return used_cabins
    

#### fill people that must be sharing a room with people in the same group
ie, from their chome planet we know the decks they could be in, if the side and number below and above are one above and one below then they must be sharing with someone from their group. make sure that their group has only one cabin

need to make sure is only one person  with cabin being na in the group otherwise one na might be in another cabin 

for row in dataframe

if cabin.isna() and homeplanet is known,
 for potential decks in homeplanets remit ie europa check a, b , c in dataframe up to that point and over that point, see if any have a free space and if not then fill it with the one in the same group

In [665]:
def checks(df):
    for home in homeplanets:
        print("\n" + home)
        print(df[df.HomePlanet == home].Deck.value_counts())
    return df

In [666]:
df2 = df.copy()

Finding groups that have more than 1 member that all share the same cabin and that dont have other nans in group

In [714]:
df2.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 72
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  72
Number                72
Side                  72
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64

# one room available and its alone in its group ( should be first in cabin fill)

In [743]:
def one_room_available_and_alone_in_group(df):
    for index, passenger in df[df.Cabin.isna()].iterrows():
        options = []
        for deck in passenger.potential_decks:
            for side in cabin_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                
                if top_room_number_before + 1 != smallest_room_number_after:
                    if pd.isna(top_room_number_before) and smallest_room_number_after == 0:
                        continue
                    elif top_room_number_before == smallest_room_number_after:
                        continue
                    else:
                        options.append([deck,side,top_room_number_before,smallest_room_number_after])
        
        if len(options) == 1:
            option = options[0]
            if option[2] + 2 == option[3] and len(df[df.Group == passenger.Group]) == 1:
                df.loc[index,'Cabin'] = str(option[0]) + "/" + str(int(option[2] + 1)) + "/" + str(option[1])
    
    df[['Deck','Number','Side']] = df.apply(fill_deck_cabin_side,axis = 1)
    return df

df2 = one_room_available_and_alone_in_group(df2)

                
    
    

    

In [744]:
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Group,GroupNumber,Deck,Number,Side,FirstName,LastName,GroupSize,Bills,potential_decks
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,1,1,B,0.0,P,Maham,Ofracculy,1,0.0,None
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,2,1,F,0.0,S,Juanna,Vines,1,736.0,None
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,3,1,A,0.0,S,Altark,Susent,2,10383.0,None
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,...,3,2,A,0.0,S,Solam,Susent,2,5176.0,None
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,...,4,1,F,1.0,S,Willy,Santantines,1,1091.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,9277_01,Earth,True,G/1498/S,PSO J318.5-22,43.0,False,0.0,0.0,0.0,...,9277,1,G,1498.0,S,Lilace,Leonzaley,1,0.0,None
12966,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,...,9278,1,G,1499.0,S,Kurta,Mondalley,1,0.0,None
12967,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,...,9279,1,G,1500.0,S,Fayey,Connon,1,1873.0,None
12968,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,...,9280,1,E,608.0,S,Celeon,Hontichre,2,4637.0,None


In [745]:
df2.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 68
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  68
Number                68
Side                  68
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64

# there arent any free rooms for it so has to share


In [746]:
def no_free_rooms_so_shares(df):
    for index, passenger in df[df.Cabin.isna()].iterrows():
        options = False
        for deck in passenger.potential_decks:
            for side in cabin_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                
                if top_room_number_before + 1 != smallest_room_number_after:
                    if pd.isna(top_room_number_before) and smallest_room_number_after == 0:
                        continue
                    elif top_room_number_before == smallest_room_number_after:
                        continue
                    else:
                        options = True
                        break
            if options:
                break
        if not options:
            other_group_member = df[(df.Group == passenger.Group) & (~df.Cabin.isna()) & (df.Deck.isin(passenger.potential_decks))]
            print(index)
            if len(other_group_member.Cabin.unique()) == 1:
                
                df.loc[index,'Cabin'] = other_group_member.iloc[0].Cabin
                
    df[['Deck','Number','Side']] = df.apply(fill_deck_cabin_side,axis = 1)
    return df
  
            # Slice the DataFrame first and then apply the boolean mask

df2 = no_free_rooms_so_shares(df2)

2442
2970
4569
4751
12174


Manual imputes


In [747]:
df2.loc[4233,'Cabin'] = 'B/98/P'
df2.loc[4254,'Cabin'] = 'B/99/P'
df2.loc[6493,'Cabin'] = 'E/300/S'
df2.loc[6514,'Cabin'] = 'E/301/S'
df2.loc[12892,'Cabin'] = 'F/1785/S'
df2.loc[12893,'Cabin'] = 'F/1785/S'
df2.loc[9265,'Cabin'] = 'F/1267/S'
df2.loc[9267,'Cabin'] = 'F/1267/S'
df2[['Deck','Number','Side']] = df2.apply(fill_deck_cabin_side,axis = 1)


In [748]:
df2.isna().sum()

PassengerId            0
HomePlanet            13
CryoSleep            310
Cabin                 68
Destination          274
Age                  270
VIP                  296
RoomService          263
FoodCourt            289
ShoppingMall         306
Spa                  284
VRDeck               268
Name                 294
Set                    0
Transported         4277
Group                  0
GroupNumber            0
Deck                  68
Number                68
Side                  68
FirstName            294
LastName             294
GroupSize              0
Bills                785
potential_decks    12671
dtype: int64

Cases
* there arent any free rooms for it so has to share
* its the only person that could fill that room 

# free room where only one person can take it

In [749]:
def rooms_to_fill(df):
    rooms = {}
    for deck in cabin_decks:
        for side in cabin_sides:
            rooms_seen = list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())
            largest_room_number = int(max(list(df[(df.Deck == deck) & (df.Side == side)].dropna(subset = ['Number']).Number.unique())))
            for i in range(largest_room_number):
                if i not in rooms_seen:
                    if deck not in rooms:
                        rooms[deck] = {'P':[],'S':[]}
                    rooms[deck][side].append(i)
    return rooms

rooms = rooms_to_fill(df2)
                    
                    
                    
            

In [750]:
def fill_by_empty_room(df):
    for deck in rooms.keys():
        for side in ['P','S']:
            for number in rooms[deck][side]:
                indices_matching = []
                temp = df[(df.Cabin.isna())]
                for index,row in temp.iterrows():
                    if deck not in row.potential_decks:
                        continue
                            
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    if max(list(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique())) < number:
                        if min(list(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())) > number:
                            indices_matching.append(index)
                if len(indices_matching) == 1:
                    df.loc[indices_matching[0],'Cabin'] = deck + "/" + str(number) + "/" + side
                    df.loc[indices_matching[0],['Side','Deck','Number']] = [side,deck,number]
fill_by_empty_room(df2)

In [751]:
df2[df2.Cabin.isna()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Group,GroupNumber,Deck,Number,Side,FirstName,LastName,GroupSize,Bills,potential_decks
404,0293_01,Europa,True,NaN,TRAPPIST-1e,47.0,False,0.0,0.0,0.0,...,293,1,NaN,NaN,NaN,Tauxon,Suptibler,1,0.0,"[A, B, C, D, E, T]"
421,0310_01,Europa,False,NaN,TRAPPIST-1e,67.0,False,NaN,230.0,0.0,...,310,1,NaN,NaN,NaN,Naviton,Coudered,1,NaN,"[A, B, C, D, E, T]"
479,0348_02,Mars,NaN,NaN,TRAPPIST-1e,36.0,False,520.0,0.0,1865.0,...,348,2,NaN,NaN,NaN,Weet,Mane,2,2385.0,"[D, E, F]"
505,0364_02,Mars,False,NaN,TRAPPIST-1e,37.0,False,731.0,0.0,517.0,...,364,2,NaN,NaN,NaN,Anakes,Chité,2,1298.0,"[D, E, F]"
517,0374_02,Earth,False,NaN,TRAPPIST-1e,36.0,False,6.0,0.0,0.0,...,374,2,NaN,NaN,NaN,Tamie,Sterreray,2,789.0,"[E, F, G]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12241,8772_02,Europa,False,NaN,55 Cancri e,53.0,False,0.0,1127.0,0.0,...,8772,2,NaN,NaN,NaN,Naosura,Motled,2,NaN,"[A, B, C, D, E, T]"
12651,9057_01,Europa,False,NaN,55 Cancri e,36.0,True,132.0,3479.0,0.0,...,9057,1,NaN,NaN,NaN,Coxan,Statch,2,7397.0,"[A, B, C, D, E, T]"
12658,9062_02,Earth,True,NaN,TRAPPIST-1e,65.0,False,0.0,0.0,0.0,...,9062,2,NaN,NaN,NaN,Lerion,Pughlinsons,2,0.0,"[E, F, G]"
12668,9069_03,Europa,True,NaN,55 Cancri e,25.0,False,0.0,0.0,0.0,...,9069,3,NaN,NaN,NaN,Bath,Brakeng,5,0.0,"[A, B, C, D, E, T]"


# remaining cabins

# which cabin for empty people

In [752]:
def remaining_cabins(df):
    for index, passenger in df[df.Cabin.isna()].iterrows():
        print("\nindex", index)
        print("passenger",passenger.PassengerId)
        print("GroupSize", passenger.GroupSize)
        options = []
        for deck in decks_by_planet[passenger.HomePlanet]:
            for side in cabin_sides:
                
                before_slice = df.iloc[:index]
                after_slice = df.iloc[index+1:] 
                top_room_number_before = np.max(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number)
                smallest_room_number_after = np.min(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number)
                
                if top_room_number_before + 1 != smallest_room_number_after:
                    if pd.isna(top_room_number_before) and smallest_room_number_after == 0:
                        continue
                    elif top_room_number_before == smallest_room_number_after:
                        continue
                    else:
                        options.append([deck,side,top_room_number_before,smallest_room_number_after])
        print(options)


# which people for empty cabin

In [753]:
def fill_by_empty_room(df):
    for deck in rooms.keys():
        for side in ['P','S']:
            for number in rooms[deck][side]:
                indices_matching = []
                print()
                print('deck',deck,'side',side,'number',number)
                temp = df[(df.Cabin.isna()) & (pd.isna(df.HomePlanet) | df.HomePlanet.isin(planet_by_deck[deck]))]
                for index,row in temp.iterrows():
                    before_slice = df.iloc[:index]
                    after_slice = df.iloc[index+1:] 
                    if max(list(before_slice[(before_slice.Deck == deck) & (before_slice.Side == side)].Number.unique())) < number:
                        if min(list(after_slice[(after_slice.Deck == deck) & (after_slice.Side == side)].Number.unique())) > number:
                            indices_matching.append(index)
                print(df.iloc[indices_matching].PassengerId)
                print()
                print()


## what can be deduced with these cabins now filled ?

In [754]:
remaining_cabins(df2)


index 404
passenger 0293_01
GroupSize 1
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

index 421
passenger 0310_01
GroupSize 1
[['B', 'P', 12.0, 14.0], ['C', 'S', 12.0, 14.0]]

index 479
passenger 0348_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]

index 505
passenger 0364_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]

index 517
passenger 0374_02
GroupSize 2
[['E', 'P', 19.0, 22.0]]

index 1401
passenger 0992_04
GroupSize 6
[['E', 'P', 57.0, 59.0]]

index 1423
passenger 1006_03
GroupSize 3
[['E', 'P', 57.0, 59.0]]

index 1429
passenger 1011_01
GroupSize 2
[['E', 'P', 57.0, 59.0]]

index 1466
passenger 1041_01
GroupSize 1
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0], ['E', 'P', 57.0, 59.0]]

index 1543
passenger 1095_01
GroupSize 1
[['C', 'S', 39.0, 41.0], ['D', 'S', 35.0, 37.0]]

index 2442
passenger 1709_03
GroupSize 7
[]

index 2970
passenger 2092_03
GroupSize 5
[]

index 3529
passenger 2513_01
GroupSize 1
[['E', 'P', 149.0, 151.0], ['F', 'P', 518.0, 520.0]]

index 3530
passenger 2514_01


[['G', 'P', 589.0, 591.0], ['G', 'S', 578.0, 580.0]]

index 5017
passenger 3599_01
GroupSize 1
[['G', 'P', 589.0, 591.0], ['G', 'S', 578.0, 580.0]]

index 5857
passenger 4181_03
GroupSize 6
[['T', 'P', 4.0, nan]]

index 6075
passenger 4336_03
GroupSize 4
[['T', 'P', 4.0, nan]]

index 6133
passenger 4378_02
GroupSize 2
[['T', 'P', 4.0, nan]]

index 6472
passenger 4625_03
GroupSize 7
[['T', 'P', 4.0, nan]]

index 6535
passenger 4662_03
GroupSize 3
[['T', 'P', 4.0, nan]]

index 6578
passenger 4690_02
GroupSize 2
[['T', 'P', 4.0, nan]]

index 6709
passenger 4782_01
GroupSize 5
[['T', 'P', 4.0, nan]]

index 6927
passenger 4953_01
GroupSize 1
[['C', 'S', 192.0, 194.0], ['T', 'P', 4.0, nan]]

index 6952
passenger 4974_02
GroupSize 2
[['C', 'S', 192.0, 194.0], ['T', 'P', 4.0, nan]]

index 7219
passenger 5164_02
GroupSize 2
[['T', 'P', 4.0, nan]]

index 7333
passenger 5254_03
GroupSize 4
[['T', 'P', 4.0, nan]]

index 7638
passenger 5480_01
GroupSize 1
[['C', 'S', 213.0, 215.0], ['T', 'P', 4.0, 

deck a needs one deck b needs 3 all p d needs 3 p
deck c needs 6 s
deck e needs 5 p and 2 s
deck f needs 4 p and 2 s
deck g needs 4 and 4

In [755]:
df2.loc[9265,'Cabin'] = 'F/1267/S'
df2.loc[9267,'Cabin'] = 'F/1267/S'

In [756]:
fill_by_empty_room(df2)



deck B side P number 13
404    0293_01
421    0310_01
Name: PassengerId, dtype: object



deck F side P number 519
3529    2513_01
3530    2514_01
Name: PassengerId, dtype: object



deck F side P number 1489
10081    7182_01
10082    7183_01
Name: PassengerId, dtype: object



deck F side P number 1544
10434    7463_01
10440    7469_01
Name: PassengerId, dtype: object



deck F side S number 1424
10394    7429_01
10408    7440_01
10411    7442_02
10434    7463_01
Name: PassengerId, dtype: object



deck A side P number 94


12651    9057_01
12668    9069_03
Name: PassengerId, dtype: object



deck G side P number 590
5016    3598_01
5017    3599_01
Name: PassengerId, dtype: object



deck G side P number 1157
10081    7182_01
10082    7183_01
Name: PassengerId, dtype: object



deck G side P number 1286
11074    7942_02
11082    7948_01
Name: PassengerId, dtype: object



deck G side P number 1474
12658    9062_02
12671    9070_01
Name: PassengerId, dtype: object



deck G side S number 579
5016    3598_01
5017    3599_01
Name: PassengerId, dtype: object



deck G side S number 1206
10408    7440_01
10411    7442_02
Name: PassengerId, dtype: object



deck G side S number 1212
10434    7463_01
10440    7469_01
Name: PassengerId, dtype: object



deck G side S number 1282
11074    7942_02
11082    7948_01
Name: PassengerId, dtype: object



deck C side S number 13
404    0293_01
421    0310_01
Name: PassengerId, dtype: object



deck C side S number 40
1466    1041_01
1543    1095_01
Name: PassengerId, dty